In [ ]:
# General import of torch.
import torch
# Import for graph blocks of torch.
import torch.nn as nn
# Import the models library, to get the model to be used.
import torchvision.models as models
# Import optim library, to get the optimizer to be used.
import torch.optim as optim
# Import torchvision, to manage the input data.
import torchvision
# To apply transformations to the data (when loaded).
import torchvision.transforms as transforms
# To calculate softmax.
from torch.nn import Softmax
softmax = Softmax(dim=1)

# Metrics zero_division
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

# General imports.
import os
import time
import wandb
import random
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from pathlib import Path

### Enviroment configuration.

In [ ]:
# Input size of the model.
inputSize = 'inputSize'
# Output size of the model.
outputSize = 'outputSize'
# Batch size.
batchSize = 'batchSize'
# Epochs amount.
epochs = 'epochs'
# Learning rate.
learningRate = 'learningRate'
# Class names.
classes = 'classes'
# Class IDs.
classesIDs = 'classesIDs'
# Number of classes to classify.
classesLen = 'classesLen'

config = {
    inputSize    : 224,
    outputSize   : 39,
    batchSize    : 128,
    epochs       : 200,
    learningRate : 0.001,
    classes : [
        'Apple - Apple scab',
        'Apple - Black rot',
        'Apple - Cedar apple rust',
        'Apple - Healthy',
        'Background without leaves',
        'Blueberry - Healthy',
        'Cherry - Healthy',
        'Cherry - Powdery mildew',
        'Corn - Cercospora',
        'Corn - Common rust',
        'Corn - Healthy',
        'Corn - Northern Leaf Blight',
        'Grape - Black rot',
        'Grape - Esca',
        'Grape - Healthy',
        'Grape - Leaf blight',
        'Orange - Haunglongbing',
        'Peach - Bacterial spot',
        'Peach - Healthy',
        'Pepper bell - Bacterial spot',
        'Pepper bell - healthy',
        'Potato - Early blight',
        'Potato - Healthy',
        'Potato - Late blight',
        'Raspberry - healthy',
        'Soybean - Healthy',
        'Squash - Powdery mildew',
        'Strawberry - Healthy',
        'Strawberry - Leaf scorch',
        'Tomato - Bacterial spot',
        'Tomato - Early blight',
        'Tomato - Healthy',
        'Tomato - Late blight',
        'Tomato - Leaf Mold',
        'Tomato - Septoria leaf spot',
        'Tomato - Spider mites',
        'Tomato - Target Spot',
        'Tomato - Mosaic virus',
        'Tomato - Yellow Leaf Curl Virus'
    ],
    classesIDs : [i for i in range(39)],
    classesLen : 39
}

# Device to be used, prefer cuda, if available.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Miscellaneous functions.

In [ ]:
# About metrics.
# Metric dictionary keys.
# For preprocessing.
_loss        = 'Loss'
_groundtruth = 'Groundtruth'
_logits      = 'Logits'
# For postprocessing.
_probabilities   = 'Probabilities'
_predictions     = 'Predictions'
_accuracyClass   = 'Accuracy class'
_accuracy        = 'Accuracy'
_recall          = 'Recall'
_precision       = 'Precision'
_f1              = 'F1'
_auc             = 'AUC'
# For torch save
_model           = 'Model State Dic'
_optimizer       = 'Optimizer State Dic'
_epoch           = 'Epoch'
_metricsTrain    = 'Resulting metrics (training)'
_metricsTest     = 'Resulting metrics (testing)'

_metricsPrint = [_accuracy, _recall, _precision, _f1, _auc]

# Get a clean dictionary for the metrics.
def getMetricsDict():
    return {
        _loss          : torch.tensor(0.),
        _groundtruth   : torch.tensor([]),
        _logits        : torch.tensor([])
    }

# Function used to update the dictionary of resulting metrics.
def updateRunningMetrics(logits, groundtruth, loss, batchAmount, metricsResults):
    # Accumulate the loss.
    metricsResults[_loss] += loss.cpu() / batchAmount
    # Accumulate the groundtruth and the logits.
    metricsResults[_groundtruth] = torch.cat((metricsResults[_groundtruth], groundtruth.cpu())) 
    metricsResults[_logits] = torch.cat((metricsResults[_logits], logits.cpu()))

# Function used to process the dictionary of resulting metrics (make final calculations).
def processRunningMetrics(metricsResults):
    # Detach the other values in the dictionary.
    metricsResults[_loss] = metricsResults[_loss].detach()
    metricsResults[_groundtruth] = metricsResults[_groundtruth].detach()
    metricsResults[_logits] = metricsResults[_logits].detach()
    # Save in the dictionary the probabilities and the predictions.
    metricsResults[_probabilities] = softmax(metricsResults[_logits]).detach()
    metricsResults[_predictions] = torch.argmax(metricsResults[_probabilities], axis=1).detach()

    # Get Groundtruth (as numpy).
    groundtruth = metricsResults[_groundtruth].detach().numpy()
    # Get probabilities (as numpy).
    probs = metricsResults[_probabilities].detach().numpy()
    # Get predictions (as numpy).
    preds = metricsResults[_predictions].detach().numpy()

    # Calculate accuracy by class.
    confusionMatrix = confusion_matrix(groundtruth, preds)
    confusionMatrix = confusionMatrix.astype('float') / confusionMatrix.sum(axis=1)[:, np.newaxis]
    metricsResults[_accuracyClass] = torch.tensor(confusionMatrix.diagonal())

    # Calculate accuracy.
    metricsResults[_accuracy] = torch.tensor(accuracy_score(groundtruth, preds))
    # Calculate recall.
    metricsResults[_recall] = torch.tensor(recall_score(groundtruth, preds, average='macro', zero_division=0))
    # Calculate precision.
    metricsResults[_precision] = torch.tensor(precision_score(groundtruth, preds, average='macro', zero_division=0))
    # Calculate F1.
    metricsResults[_f1] = torch.tensor(f1_score(groundtruth, preds, average='macro', zero_division=0))
    # Calculate AUC.
    metricsResults[_auc] = torch.tensor(roc_auc_score(groundtruth, probs, multi_class='ovr'))

# Pretty print the metrics dictionaries.
def printMetricsDict(metricsResults):
    # All metrics to print
    metricPrints = []

    # Format the loss.
    lossPrint = 'Loss: {:.4f}'.format(metricsResults[_loss])
    metricPrints.append(lossPrint)

    # Format the the remaining metrics.
    baseMetricString = '{}: {:1.4f}'
    for metric in _metricsPrint:
        metricPrints.append(baseMetricString.format(metric, metricsResults[metric]))

    print(', '.join(metricPrints))

# This functions process an metrics result dictionary for wandb. Is necessary to indicte
#   the metrics origin, training or testing.
def processMetricsWandb(metricsResults, training=False):
    # Get the prefix to log on wandb, the keys must be different.
    resultsType = 'training' if training else 'testing'

    _confusionMatrix = 'Confusion matrix'

    # All the wandb keys are based in the original metrics results keys.
    lossKey = '{} ({})'.format(_loss, resultsType)
    metricsKeys = ['{} ({})'.format(_metric, resultsType) for _metric in _metricsPrint]
    accuracyClassKeys = ['{} accuracy ({})'.format(_class, resultsType) for _class in config[classes]]
    confusionMatrixKey = '{} ({})'.format(_confusionMatrix, resultsType)

    # Get the confusion matrix
    confusionMatrix = wandb.plot.confusion_matrix(y_true=metricsResults[_groundtruth].numpy(),
        preds=metricsResults[_predictions].numpy(), class_names=config[classes], title=confusionMatrixKey)

    # Make the dictionary for wandb and store the values.
    wandbDict = {
        lossKey            : metricsResults[_loss].item(),
        confusionMatrixKey : confusionMatrix
    }
    for i in range(len(metricsKeys)):
        wandbDict[metricsKeys[i]] = metricsResults[_metricsPrint[i]].item()
    for i in range(config[classesLen]):
        wandbDict[accuracyClassKeys[i]] = metricsResults[_accuracyClass][i].item()

    # Return, to log later.
    return wandbDict

# Get the metrics dictionaries for wandb and log them.
def logMetricsWandb(trainMetricsResults, testMetricsResults):
    # Get both dictionaries for wandb.
    wandbTrainDict = processMetricsWandb(trainMetricsResults, training=True)
    wandbTestDict  = processMetricsWandb(testMetricsResults, training=False)

    # Merge the dictionaries.
    wandbDict = {**wandbTrainDict, **wandbTestDict}

    # Log on wandb
    wandb.log(wandbDict)

# Function used to save the model and the metrics.
def saveEpochData(trainMetricsResults, testMetricsResults, model, optimizer, epoch, rootPath):
    # Create a dir for the current epoch.
    runDir = os.path.join(os.getcwd(), rootPath, str(epoch))
    Path(runDir).mkdir(parents=True, exist_ok=True)

    # Path
    savePath = os.path.join(runDir, 'model.pth')

    # Make dict for torch.save
    saveDict = {
        _model     : model.state_dict(),
        _optimizer : optimizer.state_dict(),
        _epoch     : epoch
    }

    # Save both metrics, for train and test.
    metricsResults = {
        _metricsTrain : trainMetricsResults,
        _metricsTest  : testMetricsResults
    }

    # Merge the save dict with the metricsResults dict.
    saveDict = {**metricsResults, **saveDict}

    # Save
    torch.save(saveDict, savePath)

### Loader function.
Should return the training loader and test loader, a iterable object by batches.

In [ ]:
# Transformation definitions.
transformTrain = transforms.Compose([
        transforms.RandomResizedCrop(config[inputSize]),  # This one does a resize (it cuts randomly, it doesn't keep the whole image).
        transforms.RandomHorizontalFlip(),                # Flip the image horizontally randomly.
        transforms.ToTensor(),                            # Make the image a tensor.
        transforms.Normalize(
            [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Expected normalization for pretrained pytorch models.
    ])
transformTest = transforms.Compose([
        transforms.Resize(config[inputSize]),             # Resize the image, keeping all pixels.
        transforms.CenterCrop(config[inputSize]),         # Cut the image in the center.
        transforms.ToTensor(),                            # Make the image a tensor.
        transforms.Normalize(
            [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Expected normalization for pretrained pytorch models.
    ])

# Function used to get the data loaders.
# A folder with two folders inside called train and test is expected as a rootPath.
def getLoaders(rootPath):
    trainPath = os.path.join(rootPath, 'labelTrain')
    testPath  = os.path.join(rootPath, 'labelTest')

    # Get the training and test data, apply the transformations.
    trainset = torchvision.datasets.ImageFolder(root=trainPath, transform=transformTrain)
    testset  = torchvision.datasets.ImageFolder(root=testPath,  transform=transformTest)

    # Get the loaders, to iterate the data through batches.
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=config[batchSize], shuffle=True, num_workers=2)
    testloader  = torch.utils.data.DataLoader(testset,  batch_size=config[batchSize], shuffle=True, num_workers=2)
    
    return trainloader, testloader

### Training method.
This method takes care of a single training pass. Another function call this one multiple times.

In [ ]:
def trainEpoch(dataloader, model, criterion, optimizer):
    # Metrics for training.
    metricsResults = getMetricsDict()

    # Enable the grad, for training.
    with torch.set_grad_enabled(True):

        # Indicate that the model is going to be trained.
        model.train()

        # Loader len, for metrics calculation.
        loaderLen = len(dataloader)

        # Iterate the batches for training.
        for batch in dataloader:
            # Train the model.
            # Get the inputs and labels, and move them to the selected device.
            inputs, labels = batch[0].to(device), batch[1].to(device)
            # Zero the gradient parameters.
            optimizer.zero_grad()
            # Get the predictions.
            outputs = model(inputs)
            # Calculate the error.
            loss = criterion(outputs, labels)
            # Calculates the derivatives of the parameters that have a gradient.
            loss.backward()
            # Update the parameters based on the computer gradient.
            optimizer.step()
            # Metrics for the training set.
            updateRunningMetrics(outputs, labels, loss, loaderLen, metricsResults)

    return metricsResults

### Evaluation method.
This method evaluates the model for a specified dataset.

In [ ]:
def evaluate(dataloader, model, criterion):
    # Metrics for testing.
    metricsResults = getMetricsDict()

    # Enable the grad, for training.
    with torch.set_grad_enabled(False):

        # Indicate that the model is going to be evaluated.
        model.eval()

        # Loader len, for metrics calculation.
        loaderLen = len(dataloader)

        # Iterate the batches for testing.
        for batch in dataloader:
            # Test the model.
            # Get the inputs and labels, and move them to the selected device.
            inputs, labels = batch[0].to(device), batch[1].to(device)
            # Get the predictions.
            outputs = model(inputs)
            # Calculate the error.
            loss = criterion(outputs, labels)
            # Metrics for the testing set.
            updateRunningMetrics(outputs, labels, loss, loaderLen, metricsResults)

    return metricsResults

### Trainining and evaluate method.
For the specific purpose of this project, in each epoch we evaluate metrics for each data set (training and testing) in each epoch, this method simplifies the process. 

In [ ]:
def trainAndEvaluate(trainloader, testloader, model, criterion, optimizer, savePath):

    startTimeTotal = time.time()

    for epoch in range(1, config[epochs] + 1):
        
        # Train the model.
        trainMetricsResults = trainEpoch(trainloader, model, criterion, optimizer)
        processRunningMetrics(trainMetricsResults)

        # Evaluate the model.
        testMetricsResults = evaluate(testloader, model, criterion)
        processRunningMetrics(testMetricsResults)

        # Log on wandb
        logMetricsWandb(trainMetricsResults, testMetricsResults)

        # Save model and metrics for the epochs.
        saveEpochData(trainMetricsResults, testMetricsResults, model, optimizer, epoch, savePath)

        # Print the results.
        if epoch % 5 == 0:
            print('**', '[', 'Epoch ', epoch, ']', '*' * 48, sep='')
            print('\tTraining results:', end=' ')
            printMetricsDict(trainMetricsResults)
            print('\t Testing results:', end=' ')
            printMetricsDict(testMetricsResults)
        
    # Print time
    print('Epochs terminados')
    print("--- %s seconds ---" % (time.time() - startTimeTotal))

### General method for execution.

In [ ]:
def executeTest(net, dataPath, runName, savePath):
    # Get criterion and optimizer.
    # Optimizer and the loss funtion used to train the model.
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adagrad(net.parameters(), lr=config[learningRate])

    # Get the loaders.
    trainloader, testloader = getLoaders(dataPath)

    # Init wandb
    run = wandb.init(project='Classifier-UNET-RESNET', entity='tecai', config=config, name=runName)

    # Train and evaluate
    trainAndEvaluate(trainloader, testloader, net, criterion, optimizer, savePath)

    # Finish wandb
    run.finish()

# Experiments
The experiments seek to explore the transfer learning, so it will focus on seeing the performance of the model in different datasets, exchanging parameters learned from other runs.

### First run
For the first run we are going to explore the performance of resnet34 with pretrained parameters on the raw dataset.

In [ ]:
# Get the model.
# Get a predefined model from pytorch, without the pretrained parameters.
net = models.resnet34(pretrained=False)
# Get the input size of the las layer of the model.
llInputSize = net.fc.in_features
# Modify the last layer of the model, to classify the amount of required classes.
net.fc = nn.Linear(llInputSize, config[outputSize])
# Load the model to the selected device.
net.to(device)

# Paths of data.
dataPath = 'data/corrida1'
savePath = 'runs/cnn(corrida1)'

# Run name (for wanbd).
runName = 'CNN (corrida 1)'

# Execute.
executeTest(net, dataPath, runName, savePath)

In [ ]:
# Get the model.
# Get a predefined model from pytorch, without the pretrained parameters.
net = models.resnet34(pretrained=False)
# Get the input size of the las layer of the model.
llInputSize = net.fc.in_features
# Modify the last layer of the model, to classify the amount of required classes.
net.fc = nn.Linear(llInputSize, config[outputSize])
# Load the model to the selected device.
net.to(device)

# Paths of data.
dataPath = 'data/corrida2'
savePath = 'runs/cnn(corrida2)'

# Run name (for wanbd).
runName = 'CNN (corrida 2)'

# Execute.
executeTest(net, dataPath, runName, savePath)